In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim.lr_scheduler as lr_scheduler

In [3]:
# Download training data from open datasets.
# Guassian blur - how to do: RESEARCH
training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.RandomHorizontalFlip(),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                    transforms.RandomVerticalFlip(), transforms.RandomRotation(45),
                                    transforms.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1))]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

# Download test data from open datasets.
val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

100%|██████████| 344862509/344862509 [00:17<00:00, 19924775.17it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 458224.29it/s]


100%|██████████| 14989/14989 [00:00<00:00, 28217424.89it/s]


In [4]:
batch_size = 32
num_classes = 102
learning_rate = 0.001
num_epochs = 500

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

In [5]:
import torch.nn.functional as F
import os

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.batch_layer1 = nn.BatchNorm2d(num_features=64) 
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.batch_layer2 = nn.BatchNorm2d(num_features=128) 
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.batch_layer3 = nn.BatchNorm2d(num_features=256)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)
        self.batch_layer4 = nn.BatchNorm2d(num_features=512)
        self.max_pool4 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(2048 , 1024)
        self.dropout = nn.Dropout(0.25)
        self.relu1 = nn.PReLU()
        self.fc2 = nn.Linear(1024, num_classes)
    

    def forward(self, x):
        #print(str(x.size()))
        out = self.conv_layer1(x)
        out = self.batch_layer1(out)
        out = self.relu1(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer2(out)
        out = self.batch_layer2(out)
        out = self.relu1(out)
        out = self.max_pool2(out)
        
        out = self.conv_layer3(out)
        out = self.batch_layer3(out)
        out = self.relu1(out)
        out = self.max_pool3(out)

        out = self.conv_layer4(out)
        out = self.batch_layer4(out)
        out = self.relu1(out)
        out = self.max_pool4(out)
              
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

net = Net(num_classes)

def save_network(network, epoch_label):
    save_filename = 'net_%s.pth' % epoch_label
    save_path = os.path.join(save_filename)
    torch.save(network.state_dict(), save_path)
    
def testValidationData(network):
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return (100 * correct / total)

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
saveCounter = 0
minLoss = 10000
valAccurary = 0

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
      
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    if minLoss > loss and epoch % 5 == 0:
        tempAcc = testValidationData(net)
        if tempAcc > valAccurary:
            valAccurary = tempAcc
            saveCounter += 1
            save_network(net, saveCounter)
            print("SAVE COMPLETE NEW ACC:", valAccurary)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

SAVE COMPLETE NEW ACC: 1.4705882352941178
Epoch [1/500], Loss: 4.7370
Epoch [2/500], Loss: 4.5019
Epoch [3/500], Loss: 4.2382
Epoch [4/500], Loss: 4.2971
Epoch [5/500], Loss: 4.1987
SAVE COMPLETE NEW ACC: 8.529411764705882
Epoch [6/500], Loss: 3.9264
Epoch [7/500], Loss: 3.8610
Epoch [8/500], Loss: 4.0014
Epoch [9/500], Loss: 3.9418
Epoch [10/500], Loss: 3.6513
SAVE COMPLETE NEW ACC: 14.117647058823529
Epoch [11/500], Loss: 3.5700
Epoch [12/500], Loss: 3.7523
Epoch [13/500], Loss: 3.7487
Epoch [14/500], Loss: 3.3074
Epoch [15/500], Loss: 3.2793
SAVE COMPLETE NEW ACC: 18.431372549019606
Epoch [16/500], Loss: 3.3676
Epoch [17/500], Loss: 3.4928
Epoch [18/500], Loss: 3.0209
Epoch [19/500], Loss: 3.1591
Epoch [20/500], Loss: 3.2735
SAVE COMPLETE NEW ACC: 19.901960784313726
Epoch [21/500], Loss: 2.9954
Epoch [22/500], Loss: 3.1055
Epoch [23/500], Loss: 3.0118
Epoch [24/500], Loss: 2.4528
Epoch [25/500], Loss: 2.8765
SAVE COMPLETE NEW ACC: 21.274509803921568
Epoch [26/500], Loss: 2.5904
Epoc

In [20]:
net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 6149 train images: 27.874451130265083 %


In [21]:
save_network(net, "FINALEPOCH")